In [5]:
import ee

# Trigger the authentication flow.
# ee.Authenticate()

ee.Initialize(project='ee-arzaaan789')

In [6]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv("TGB_dormice.csv", delimiter='\t')
df = df[df["occurrenceStatus"] == "PRESENT"]
df = df[df['year']>=2022]
# df['eventDate'] = df['eventDate'].str.replace('/','')
df['eventDate'] = pd.to_datetime(df['eventDate'], format='%Y-%m-%d', errors='coerce')
df = df[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
df = df.dropna().reset_index(drop=True)

In [7]:
import ee
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm

# ee.Initialize()

# Your dataframe 'df' must have columns: decimalLongitude, decimalLatitude
# Example: df = pd.read_csv("Erinaceus europaeus.csv", delimiter='\t')

def create_aoi(lon, lat, box_size_km=1):
    """Create approx 1km x 1km square polygon around lon, lat."""
    half_side_deg = box_size_km / 111.32 / 2  # Rough approx degrees per km

    coords = [
        [lon - half_side_deg, lat - half_side_deg],
        [lon + half_side_deg, lat - half_side_deg],
        [lon + half_side_deg, lat + half_side_deg],
        [lon - half_side_deg, lat + half_side_deg],
        [lon - half_side_deg, lat - half_side_deg]
    ]
    return ee.Geometry.Polygon(coords)

def compute_all_indices(feature, start_date, end_date):
    """Compute spectral indices and LST for one EE Feature (with AOI geometry)."""
    aoi = feature.geometry()

    s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
          .filterBounds(aoi)
          .filterDate(start_date, end_date)
          .sort('CLOUDY_PIXEL_PERCENTAGE')
          .first())
    s2 = ee.Image(s2).clip(aoi)

    blue = s2.select('B2')
    green = s2.select('B3')
    red = s2.select('B4')
    nir = s2.select('B8')
    swir = s2.select('B11')

    L = 0.5  # SAVI constant

    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI')
    ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI')
    savi = nir.subtract(red).divide(nir.add(red).add(L)).multiply(1 + L).rename('SAVI')
    mndwi = green.subtract(swir).divide(green.add(swir)).rename('MNDWI')
    ndsi = green.subtract(swir).divide(green.add(swir)).rename('NDSI')
    bsi = (red.add(blue).subtract(nir.add(swir))).divide(red.add(blue).add(nir).add(swir)).rename('BSI')
    ui = nir.subtract(swir).divide(nir.add(swir)).rename('UI')

    reducers = ee.Reducer.mean()
    scale_10m = 10

    ndvi_mean = ndvi.reduceRegion(reducers, aoi, scale_10m).get('NDVI')
    ndwi_mean = ndwi.reduceRegion(reducers, aoi, scale_10m).get('NDWI')
    ndbi_mean = ndbi.reduceRegion(reducers, aoi, scale_10m).get('NDBI')
    savi_mean = savi.reduceRegion(reducers, aoi, scale_10m).get('SAVI')
    mndwi_mean = mndwi.reduceRegion(reducers, aoi, scale_10m).get('MNDWI')
    ndsi_mean = ndsi.reduceRegion(reducers, aoi, scale_10m).get('NDSI')
    bsi_mean = bsi.reduceRegion(reducers, aoi, scale_10m).get('BSI')
    ui_mean = ui.reduceRegion(reducers, aoi, scale_10m).get('UI')

    # MODIS LST dataset
    modis = (ee.ImageCollection("MODIS/061/MOD11A1")
             .filterBounds(aoi)
             .filterDate(start_date, end_date)
             .select('LST_Day_1km'))

    lst_mean_img = modis.mean().multiply(0.02).clip(aoi)

    lst_mean = lst_mean_img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=1000
    ).get('LST_Day_1km')

    return feature.set({
        'NDVI': ndvi_mean,
        'NDWI': ndwi_mean,
        'NDBI': ndbi_mean,
        'SAVI': savi_mean,
        'MNDWI': mndwi_mean,
        'NDSI': ndsi_mean,
        'BSI': bsi_mean,
        'UI': ui_mean,
        'LST': lst_mean
    })

# Split df into batches
batch_size = 50
batches = [df.iloc[i:i+batch_size] for i in range(0, len(df), batch_size)]

results_list = []

index_names = ['NDVI', 'NDWI', 'NDBI', 'SAVI', 'MNDWI', 'NDSI', 'BSI', 'UI', 'LST']

for batch in tqdm(batches, desc="Processing batches"):
    features = []
    batch_indices = []
    feature_metadata = {}

    for idx, row in batch.iterrows():
        
        aoi = create_aoi(row['decimalLongitude'], row['decimalLatitude'])
        feature = ee.Feature(aoi).set('index', idx)
        features.append(feature)
        batch_indices.append(idx)  # Save the original index
        
        event_date = row['eventDate']
        start_date = (event_date - timedelta(days=15)).strftime('%Y-%m-%d')
        end_date = (event_date + timedelta(days=15)).strftime('%Y-%m-%d')
        feature_metadata[idx] = (start_date, end_date)

    # Create a FeatureCollection from the list of features
    fc = ee.FeatureCollection(features)
    
    # Define wrapper for map to inject per-feature dates
    def map_with_dates(f):
        idx = f.get('index')
        # Use dictionary lookup to get dates for this feature
        date_dict = ee.Dictionary(ee.Dictionary(feature_metadata))
        dates = ee.List(date_dict.get(ee.Number(idx)))
        return compute_all_indices(f, dates.get(0), dates.get(1))

    try:
        result_fc = fc.map(map_with_dates)
        results = result_fc.getInfo()

        rows = []
        for f in results['features']:
            props = f['properties']
            rows.append(props)

    except Exception as e:
        print(f"Error processing batch starting at index {batch.index[0]}: {e}")
        # If there's an error, create placeholder rows with None
        rows = [{'index': i, **{name: None for name in index_names}} for i in batch_indices]

    batch_results_df = pd.DataFrame(rows).sort_values('index').reset_index(drop=True)
    results_list.append(batch_results_df)

# Concatenate all batches and sort by original index
all_results_df = pd.concat(results_list).sort_values('index').reset_index(drop=True)

# Merge with original df
df_final = pd.concat([df.reset_index(drop=True), all_results_df.drop(columns=['index'])], axis=1)

print(df_final.head())

Processing batches: 100%|██████████| 157/157 [06:39<00:00,  2.54s/it]

               species  decimalLatitude  decimalLongitude  eventDate  \
0  Apodemus sylvaticus           51.468            -0.236 2024-08-06   
1  Apodemus sylvaticus           51.487            -0.258 2024-07-09   
2  Apodemus sylvaticus           51.583            -0.187 2024-08-20   
3  Apodemus sylvaticus           54.778            -1.527 2024-07-11   
4  Apodemus sylvaticus           52.615            -0.828 2024-09-29   

        BSI         LST     MNDWI      NDBI      NDSI      NDVI      NDWI  \
0 -0.505253  301.088493 -0.381012 -0.153181 -0.381012  0.521868 -0.499912   
1 -0.595485  300.180000 -0.494832 -0.124082 -0.494832  0.562891 -0.554897   
2 -0.559838  298.636845 -0.288169 -0.326648 -0.288169  0.612911 -0.557376   
3 -0.653546  295.047920 -0.441528 -0.227270 -0.441528  0.627969 -0.595242   
4 -0.401441  286.648566 -0.288573 -0.119305 -0.288573  0.421912 -0.387299   

       SAVI        UI  
0  0.782690  0.153181  
1  0.844190  0.124082  
2  0.919068  0.326648  
3  0.941

In [8]:
df_final.to_csv("tgb_dormice_full_data.csv", index=False)

df = pd.read_csv("tgb_dormice_full_data.csv")
df['eventDate'] = pd.to_datetime(df['eventDate'], format='%Y-%m-%d', errors='coerce')
df = df.dropna().reset_index(drop=True)

In [9]:
hazel = pd.read_csv("hazel.csv", delimiter='\t')
hazel = hazel[hazel["occurrenceStatus"] == "PRESENT"]
hazel = hazel[hazel['year']>=2022]
hazel['eventDate'] = pd.to_datetime(hazel['eventDate'], format='%Y-%m-%d', errors='coerce')
hazel = hazel[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
hazel = hazel.dropna().reset_index(drop=True)

birch = pd.read_csv("birch.csv", delimiter='\t')
birch = birch[birch["occurrenceStatus"] == "PRESENT"]
birch = birch[birch['year']>=2022]
birch['eventDate'] = pd.to_datetime(birch['eventDate'], format='%Y-%m-%d', errors='coerce')
birch = birch[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
birch = birch.dropna().reset_index(drop=True)

beech = pd.read_csv("beech.csv", delimiter='\t')
beech = beech[beech["occurrenceStatus"] == "PRESENT"]
beech = beech[beech['year']>=2022]
beech['eventDate'] = pd.to_datetime(beech['eventDate'], format='%Y-%m-%d', errors='coerce')
beech = beech[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
beech = beech.dropna().reset_index(drop=True)

honeysuckle = pd.read_csv("honeysuckle.csv", delimiter='\t')
honeysuckle = honeysuckle[honeysuckle["occurrenceStatus"] == "PRESENT"]
honeysuckle = honeysuckle[honeysuckle['year']>=2022]
honeysuckle['eventDate'] = pd.to_datetime(honeysuckle['eventDate'], format='%Y-%m-%d', errors='coerce')
honeysuckle = honeysuckle[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
honeysuckle = honeysuckle.dropna().reset_index(drop=True)

oak = pd.read_csv("oak.csv", delimiter='\t')
oak = oak[oak["occurrenceStatus"] == "PRESENT"]
oak = oak[oak['year']>=2022]
oak['eventDate'] = pd.to_datetime(oak['eventDate'], format='%Y-%m-%d', errors='coerce')
oak = oak[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
oak = oak.dropna().reset_index(drop=True)

hawthorn = pd.read_csv("hawthorn.csv", delimiter='\t')
hawthorn = hawthorn[hawthorn["occurrenceStatus"] == "PRESENT"]
hawthorn = hawthorn[hawthorn['year']>=2022]
hawthorn['eventDate'] = pd.to_datetime(hawthorn['eventDate'], format='%Y-%m-%d', errors='coerce')
hawthorn = hawthorn[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
hawthorn = hawthorn.dropna().reset_index(drop=True)

/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_48370/1357841717.py:1: DtypeWarning: Columns (10,38,39,41,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  hazel = pd.read_csv("hazel.csv", delimiter='\t')
/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_48370/1357841717.py:8: DtypeWarning: Columns (10,38,39,41,46) have mixed types. Specify dtype option on import or set low_memory=False.
  birch = pd.read_csv("birch.csv", delimiter='\t')
/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_48370/1357841717.py:15: DtypeWarning: Columns (10,39,41,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  beech = pd.read_csv("beech.csv", delimiter='\t')
/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_48370/1357841717.py:22: DtypeWarning: Columns (10,38,39,41,45,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  honeysuckle = pd.read_csv("honeysuckle.csv", delimiter

In [10]:
import numpy as np
from sklearn.neighbors import BallTree

# Convert lat/lon to radians for BallTree
dormice_coords = np.deg2rad(df[['decimalLatitude', 'decimalLongitude']].values)
hazel_coords = np.deg2rad(hazel[['decimalLatitude', 'decimalLongitude']].values)
birch_coords = np.deg2rad(birch[['decimalLatitude', 'decimalLongitude']].values)
beech_coords = np.deg2rad(beech[['decimalLatitude', 'decimalLongitude']].values)
honeysuckle_coords = np.deg2rad(honeysuckle[['decimalLatitude', 'decimalLongitude']].values)
oak_coords = np.deg2rad(oak[['decimalLatitude', 'decimalLongitude']].values)
hawthorn_coords = np.deg2rad(hawthorn[['decimalLatitude', 'decimalLongitude']].values)


# Build BallTree using haversine metric
tree_hazel = BallTree(hazel_coords, metric='haversine')
tree_birch = BallTree(birch_coords, metric='haversine')
tree_beech = BallTree(beech_coords, metric='haversine')
tree_honeysuckle = BallTree(honeysuckle_coords, metric='haversine')
tree_oak = BallTree(oak_coords, metric='haversine')
tree_hawthorn = BallTree(hawthorn_coords, metric='haversine')

# Define search radius: 1 km in radians
earth_radius_km = 6371.0
radius = 1 / earth_radius_km  # 1 km in radians

# Initialize presence columns
df['hazel_presence'] = 0
df['birch_presence'] = 0
df['beech_presence'] = 0
df['honeysuckle_presence'] = 0
df['oak_presence'] = 0
df['hawthorn_presence'] = 0

# Iterate through each dormice point with tqdm for progress tracking
for i in tqdm(range(len(df))):
    point = dormice_coords[i].reshape(1, -1)
    event_month = df.loc[i, 'eventDate'].month
    event_year = df.loc[i, 'eventDate'].year

    # HAZEL
    idxs = tree_hazel.query_radius(point, r=radius)[0]
    for j in idxs:
        hazel_date = hazel.loc[j, 'eventDate']
        if hazel_date.month == event_month and hazel_date.year == event_year:
            df.at[i, 'hazel_presence'] = 1
            break  # Found at least one match, no need to check further

    # BIRCH
    idxs = tree_birch.query_radius(point, r=radius)[0]
    for j in idxs:
        birch_date = birch.loc[j, 'eventDate']
        if birch_date.month == event_month and birch_date.year == event_year:
            df.at[i, 'birch_presence'] = 1
            break
            
    # BEECH
    idxs = tree_beech.query_radius(point, r=radius)[0]
    for j in idxs:
        beech_date = beech.loc[j, 'eventDate']
        if beech_date.month == event_month and beech_date.year == event_year:
            df.at[i, 'beech_presence'] = 1
            break
            
    # HONEYSUCKLE
    idxs = tree_honeysuckle.query_radius(point, r=radius)[0]
    for j in idxs:
        honeysuckle_date = honeysuckle.loc[j, 'eventDate']
        if honeysuckle_date.month == event_month and honeysuckle_date.year == event_year:
            df.at[i, 'honeysuckle_presence'] = 1
            break
            
    # OAK
    idxs = tree_oak.query_radius(point, r=radius)[0]
    for j in idxs:
        oak_date = oak.loc[j, 'eventDate']
        if oak_date.month == event_month and oak_date.year == event_year:
            df.at[i, 'oak_presence'] = 1
            break
            
    # HAWTHORN
    idxs = tree_hawthorn.query_radius(point, r=radius)[0]
    for j in idxs:
        hawthorn_date = hawthorn.loc[j, 'eventDate']
        if hawthorn_date.month == event_month and hawthorn_date.year == event_year:
            df.at[i, 'hawthorn_presence'] = 1
            break
            
            
            
            


100%|██████████| 7786/7786 [00:01<00:00, 4357.36it/s]


In [11]:
from pyproj import Transformer
from rasterio.windows import Window
import rasterio

land_cover_map = {
    1: "Deciduous woodland",
    2: "Coniferous woodland",
    3: "Arable",
    4: "Improved grassland",
    5: "Neutral grassland",
    6: "Calcareous grassland",
    7: "Acid grassland",
    8: "Fen",
    9: "Heather",
    10: "Heather grassland",
    11: "Bog",
    12: "Inland rock",
    13: "Saltwater",
    14: "Freshwater",
    15: "Supralittoral rock",
    16: "Supralittoral sediment",
    17: "Littoral rock",
    18: "Littoral sediment",
    19: "Saltmarsh",
    20: "Urban",
    21: "Suburban"
}

# Batch coordinate transformation
transformer_ni = Transformer.from_crs("EPSG:4326", "EPSG:29903", always_xy=True)
transformer_gb = Transformer.from_crs("EPSG:4326", "EPSG:27700", always_xy=True)

coords = list(zip(df['decimalLongitude'], df['decimalLatitude']))
df['easting_ni'], df['northing_ni'] = zip(*transformer_ni.itransform(coords))
df['easting_gb'], df['northing_gb'] = zip(*transformer_gb.itransform(coords))

# Raster processing optimization
gb_raster = 'gblcm2023_10m.tif'
n_ireland_raster = 'nilcm2023_10m.tif'


def get_land_cover_class(row):
    try:
        # Try GB raster first
        with rasterio.open(gb_raster) as src:
            row_idx, col_idx = src.index(row['easting_gb'], row['northing_gb'])
            # Read a small window around the point for better performance
            window = Window(col_idx, row_idx, 1, 1)
            land_cover_class = src.read(1, window=window)[0, 0]

            if land_cover_class == 0:  # Check NI raster if GB is 0
                with rasterio.open(n_ireland_raster) as src_ni:
                    row_idx, col_idx = src_ni.index(row['easting_ni'], row['northing_ni'])
                    window = Window(col_idx, row_idx, 1, 1)
                    land_cover_class = src_ni.read(1, window=window)[0, 0]

        return land_cover_map.get(land_cover_class, "Unknown")
    except Exception as e:
        print(f"Error processing row: {e}")
        return "Unknown"


tqdm.pandas()
df['Land_cover'] = df.progress_apply(get_land_cover_class, axis=1)

 39%|███▉      | 3050/7786 [00:08<00:11, 414.96it/s]

Error processing row: index 0 is out of bounds for axis 0 with size 0


100%|██████████| 7786/7786 [00:20<00:00, 378.88it/s]


In [12]:
df = df.dropna()
df=df.drop(['easting_ni', 'northing_ni', 'easting_gb', 'northing_gb'],axis=1)
# Remove where Land_cover is Unknown
df = df[df['Land_cover'] != "Unknown"].reset_index(drop=True)
df.to_csv("tgb_dormice_final_data.csv", index=False)